In [1]:
import pandas as pd
import eurostat

In [2]:
df = eurostat.get_data_df(code="HBS_STR_T223")
df = df.rename(columns={'geo\TIME_PERIOD': 'geo', '2015':'OBS_VALUE'})


In [ ]:
unique_values_df1 = df["geo"].unique()

In [ ]:
# Keep only the rows where TIME_PERIOD is equal to 2015
df = df.loc[:, ['quantile','coicop','geo','OBS_VALUE']]


In [ ]:
df_inflation = eurostat.get_data_df(code="PRC_HICP_MANR",filter_pars={'startPeriod':"2021-03"})
df_inflation = df_inflation.rename(columns={'geo\TIME_PERIOD': 'geo'})


In [ ]:
merged_df = df.merge(df_inflation, on=["coicop", "geo"])


In [ ]:
merged_df.columns

In [ ]:
cols_to_multiply = [col for col in merged_df.columns if col.startswith('202')]
merged_df[['w' + col for col in cols_to_multiply]] = merged_df[cols_to_multiply].apply(lambda x: x * merged_df['OBS_VALUE']/1000)


In [ ]:
merged_df_5_chars = merged_df[merged_df['coicop'].str.len() == 5]


In [ ]:
columns_to_sum = [col for col in merged_df_5_chars.columns if col.startswith('w')]
                  
summed_columns = []
for col in columns_to_sum:
    new_col_name = col[1:]
    summed_columns.append(new_col_name)
    merged_df_5_chars[new_col_name] = merged_df_5_chars.groupby(['geo', 'quantile'])[col].transform('sum')

df_inflation_by_quintile = merged_df_5_chars[['geo', 'quantile'] + summed_columns].drop_duplicates()

In [3]:
merged_df_4_chars = merged_df[merged_df['coicop'].str.len() == 4]
merged_df_4_chars = merged_df_4_chars.loc[(merged_df_4_chars['geo'] == 'EU27_2020')]
summed_columns = []
for col in columns_to_sum:
    new_col_name = col[1:]
    summed_columns.append(new_col_name)
    merged_df_4_chars[new_col_name] = merged_df_4_chars.groupby(['geo', 'quantile'])[col].transform('sum')

df_inflation_by_quintile_eu27 = merged_df_4_chars[['geo', 'quantile'] + summed_columns].drop_duplicates()


NameError: name 'merged_df' is not defined

In [ ]:
df_inflation_by_quintile = pd.concat([df_inflation_by_quintile, df_inflation_by_quintile_eu27])

In [ ]:
unique_values_df2 = df_inflation_by_quintile["geo"].unique()

In [ ]:
# obtenir les éléments qui sont présents dans list1 mais pas dans list2
diff_list = list(set(unique_values_df1) - set(unique_values_df2))

# afficher les éléments différenciels
print(diff_list)

ça serait bien d'avoir pour EU27_2020 au moins. Ou alors dans l'euro area (EA)

In [ ]:
dfEA = df[df["geo"] == "EU27_2020"]